# <center>Тема 4. Линейные модели классификации и регрессии
## <center>  Практика. Идентификация пользователя с помощью логистической регрессии

Тут мы воспроизведем парочку бенчмарков нашего соревнования и вдохновимся побить третий бенчмарк, а также остальных участников. Веб-формы для отправки ответов тут не будет, ориентир – [leaderboard](https://www.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2/leaderboard) соревнования.

In [64]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from scipy.sparse import csr_matrix, hstack, vstack
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

### Краткое руководство по StandardScaler и сsr_matrix
**Пример использования StandardScaler**

In [2]:
# from sklearn.preprocessing import StandardScaler

a = [1, 2, 3 ,4 ,5]
b = [10,20 ,30 ,40,50]

fd = pd.DataFrame()
fd['a'] = a
fd['b'] = b
fd

,a,b
0,1,10
1,2,20
2,3,30
3,4,40
4,5,50


In [3]:
# col_names = fd.columns
# col_names = ['a']
col_names = ['a','b']
features = fd[col_names]

scaler = StandardScaler()
scaler.fit(features)

features = scaler.transform(features.values)
fd[col_names] = features
fd

,a,b
0,-1.414214,-1.414214
1,-0.707107,-0.707107
2,0.000000,0.000000
3,0.707107,0.707107
4,1.414214,1.414214


**Пример использования csr_matrix**

In [67]:
#from scipy.sparse import csr_matrix, hstack, vstack

a = [[0,0,3,0],
     [0,1,0,0],
     [0,0,2,0],
     [1,0,0,3]]

b = csr_matrix(a)

In [68]:
b

<4x4 sparse matrix of type '<class 'numpy.int32'>'
	with 5 stored elements in Compressed Sparse Row format>

In [69]:
b.todense()

matrix([[0, 0, 3, 0],
        [0, 1, 0, 0],
        [0, 0, 2, 0],
        [1, 0, 0, 3]], dtype=int32)

In [70]:
b.nonzero()

(array([0, 1, 2, 3, 3], dtype=int32), array([2, 1, 2, 0, 3], dtype=int32))

In [71]:
b.data

array([3, 1, 2, 1, 3], dtype=int32)

In [72]:
c = [[1],
     [2],
     [0],
     [3]]

d = hstack([b, c])

In [73]:
d.todense()

matrix([[0, 0, 3, 0, 1],
        [0, 1, 0, 0, 2],
        [0, 0, 2, 0, 0],
        [1, 0, 0, 3, 3]], dtype=int32)

In [75]:
e = [1, 2, 0, 0, 5]

d = vstack([d, e])

In [76]:
d.todense()

matrix([[0, 0, 3, 0, 1],
        [0, 1, 0, 0, 2],
        [0, 0, 2, 0, 0],
        [1, 0, 0, 3, 3],
        [1, 2, 0, 0, 5]], dtype=int32)

### 1. Загрузка и преобразование данных
Зарегистрируйтесь на [Kaggle](www.kaggle.com), если вы не сделали этого раньше, зайдите на [страницу](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования и скачайте данные. Первым делом загрузим обучающую и тестовую выборки и посмотрим на данные.

In [4]:
# загрузим обучающую и тестовую выборки
train_df = pd.read_csv('train_sessions.csv',
                       index_col='session_id')
test_df = pd.read_csv('test_sessions.csv',
                      index_col='session_id')

# # приведем колонки time1, ..., time10 к временному формату
times = ['time%s' % i for i in range(1, 11)]
train_df[times] = train_df[times].apply(pd.to_datetime)
test_df[times] = test_df[times].apply(pd.to_datetime)

# отсортируем данные по времени
train_df = train_df.sort_values(by='time1')

# посмотрим на заголовок обучающей выборки
train_df.head()

,site1,time1,site2,time2,site3,time3,site4,time4,site5,time5,...,time6,site7,time7,site8,time8,site9,time9,site10,time10,target
session_id,,,,,,,,,,,,,,,,,,,,,
21669,56,2013-01-12 08:05:57,55.0,2013-01-12 08:05:57,NaN,NaT,NaN,NaT,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
54843,56,2013-01-12 08:37:23,55.0,2013-01-12 08:37:23,56.0,2013-01-12 09:07:07,55.0,2013-01-12 09:07:09,NaN,NaT,...,NaT,NaN,NaT,NaN,NaT,NaN,NaT,NaN,NaT,0
77292,946,2013-01-12 08:50:13,946.0,2013-01-12 08:50:14,951.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:15,946.0,2013-01-12 08:50:16,...,2013-01-12 08:50:16,948.0,2013-01-12 08:50:16,784.0,2013-01-12 08:50:16,949.0,2013-01-12 08:50:17,946.0,2013-01-12 08:50:17,0
114021,945,2013-01-12 08:50:17,948.0,2013-01-12 08:50:17,949.0,2013-01-12 08:50:18,948.0,2013-01-12 08:50:18,945.0,2013-01-12 08:50:18,...,2013-01-12 08:50:18,947.0,2013-01-12 08:50:19,945.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:19,946.0,2013-01-12 08:50:20,0
146670,947,2013-01-12 08:50:20,950.0,2013-01-12 08:50:20,948.0,2013-01-12 08:50:20,947.0,2013-01-12 08:50:21,950.0,2013-01-12 08:50:21,...,2013-01-12 08:50:21,946.0,2013-01-12 08:50:21,951.0,2013-01-12 08:50:22,946.0,2013-01-12 08:50:22,947.0,2013-01-12 08:50:22,0


In [5]:
train_df.shape, test_df.shape

((253561, 21), (82797, 20))

In [6]:
train_df['target'].value_counts()

0    251264
1      2297
Name: target, dtype: int64

In [7]:
target_1 = train_df['target'].mean() * 100
print(f'{target_1:.2f}', "%")

0.91 %


В обучающей выборке содержатся следующие признаки:
    - site1 – индекс первого посещенного сайта в сессии
    - time1 – время посещения первого сайта в сессии
    - ...
    - site10 – индекс 10-го посещенного сайта в сессии
    - time10 – время посещения 10-го сайта в сессии
    - target – целевая переменная, 1 для сессий Элис, 0 для сессий других пользователей
    
Сессии пользователей выделены таким образом, что они не могут быть длиннее получаса или 10 сайтов. То есть сессия считается оконченной либо когда пользователь посетил 10 сайтов подряд либо когда сессия заняла по времени более 30 минут.

В таблице встречаются пропущенные значения, это значит, что сессия состоит менее, чем из 10 сайтов. Заменим пропущенные значения нулями и приведем признаки к целому типу. Также загрузим словарь сайтов и посмотрим, как он выглядит:

In [11]:
# приведем колонки site1, ..., site10 к целочисленному формату и заменим пропуски нулями
sites = ['site%s' % i for i in range(1, 11)]
train_df[sites] = train_df[sites].fillna(0).astype('int')
test_df[sites] = test_df[sites].fillna(0).astype('int')

# загрузим словарик сайтов
with open(r"site_dic.pkl", "rb") as input_file:
    site2id = pickle.load(input_file)
#id2site = {v:k for (k, v) in site2id.items()}
site2id['unknown'] = 0
    
# датафрейм словарика сайтов
sites_dict_df = pd.DataFrame(list(site2id.values()), 
                          index=list(site2id.keys()), 
                          columns=['site'])
print(u'всего сайтов:', sites_dict_df.shape[0])
sites_dict_df.head()

всего сайтов: 48372


,site
www.abmecatronique.com,25075
groups.live.com,13997
majeureliguefootball.wordpress.com,42436
cdt46.media.tourinsoft.eu,30911
www.hdwallpapers.eu,8104


Выделим целевую переменную и объединим выборки, чтобы вместе привести их к разреженному формату.

In [13]:
# наша целевая переменная
y_train = train_df['target']

# объединенная таблица исходных данных
full_df = pd.concat([train_df.drop('target', axis=1), test_df])

# индекс, по которому будем отделять обучающую выборку от тестовой
idx_split = train_df.shape[0]

Для самой первой модели будем использовать только посещенные сайты в сессии (но не будем обращать внимание на временные признаки). За таким выбором данных для модели стоит такая идея:  *у Элис есть свои излюбленные сайты, и чем чаще вы видим эти сайты в сессии, тем выше вероятность, что это сессия Элис и наоборот.*

Подготовим данные, из всей таблицы выберем только признаки `site1, site2, ... , site10`. Напомним, что пропущенные значения заменены нулем. Вот как выглядят первые строки таблицы:

In [14]:
# табличка с индексами посещенных сайтов в сессии
full_sites = full_df[sites]
full_sites.head()

,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
21669,56,55,0,0,0,0,0,0,0,0
54843,56,55,56,55,0,0,0,0,0,0
77292,946,946,951,946,946,945,948,784,949,946
114021,945,948,949,948,945,946,947,945,946,946
146670,947,950,948,947,950,952,946,951,946,947


Сессии представляют собой последовательность индексов сайтов и данные в таком виде неудобны для линейных методов. В соответствии с нашей гипотезой (у Элис есть излюбленные сайты) надо преобразовать эту таблицу таким образом, чтобы каждому возможному сайту соответствовал свой отдельный признак (колонка), а его значение равнялось бы количеству посещений этого сайта в сессии. Это делается в две строчки:

In [15]:
full_sites.shape

(336358, 10)

In [16]:
# последовательность с индексами
sites_flatten = full_sites.values.flatten()
sites_flatten

array([  56,   55,    0, ..., 1098, 1098, 1098])

In [17]:
sites_flatten.shape

(3363580,)

In [18]:
# искомая матрица
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0],
                                sites_flatten,
                                range(0, sites_flatten.shape[0] + 10, 10)))[:, 1:]
full_sites_sparse.shape

(336358, 48371)

In [19]:
x_train_sparse = full_sites_sparse[:idx_split]
x_test_sparse = full_sites_sparse[idx_split:]

In [20]:
x_train_sparse.shape, y_train.shape

((253561, 48371), (253561,))

Еще один плюс использования разреженных матриц в том, что для них имеются специальные реализации как матричных операций, так и алгоритмов машинного обучения, что подчас позволяет ощутимо ускорить операции за счет особенностей структуры данных. Это касается и логистической регрессии. Вот теперь у нас все готово для построения нашей первой модели.

### 2. Построение первой модели

Итак, у нас есть алгоритм и данные для него, построим нашу первую модель, воспользовавшись релизацией [логистической регрессии](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) из пакета `sklearn` с параметрами по умолчанию. Первые 90% данных будем использовать для обучения (обучающая выборка отсортирована по времени), а оставшиеся 10% для проверки качества (validation). 

**Напишите простую функцию, которая будет возвращать качество модели на отложенной выборке, и обучите наш первый классификатор**.

In [21]:
def get_auc_lr_valid(X, y, C=1.0, ratio = 0.9, seed=17):
    '''
    X, y – выборка
    ratio – в каком отношении поделить выборку
    C, seed – коэф-т регуляризации и random_state 
              логистической регрессии
    '''
    # Ваш код здесь
    
    ind = round(X.shape[0] * ratio)
    x_train = X[:ind, :]
    y_train = y[:ind]
    x_test = X[ind:, :]
    y_test = y[ind:]
    
    model = LogisticRegression(C=C, random_state=seed, solver='lbfgs',n_jobs=-1)
    model.fit(x_train, y_train)
    
    pred_proba = model.predict_proba(x_test)

    return roc_auc_score(y_test, pred_proba[:, 1])

get_auc_lr_valid(x_train_sparse, y_train)

0.9199789471713791

In [23]:
# Ваш код здесь

model = LogisticRegression(C=1, random_state=21, n_jobs=-1)
model.fit(x_train_sparse, y_train)

pred_proba = model.predict_proba(x_test_sparse)[:, 1]

In [24]:
x_train_sparse.shape, x_test_sparse.shape

((253561, 48371), (82797, 48371))

In [25]:
pred_proba.shape

(82797,)

Будем считать эту модель нашей первой отправной точкой (baseline). Для построения модели для прогноза на тестовой выборке **необходимо обучить модель заново уже на всей обучающей выборке** (пока наша модель обучалась лишь на части данных), что повысит ее обобщающую способность:

In [26]:
# функция для записи прогнозов в файл
def write_to_submission_file(predicted_labels, out_file,
                             target='target', index_label="session_id"):
    sub = pd.DataFrame({'target': pred_proba,
                    'session_id': np.arange(1, pred_proba.shape[0] + 1)})
    sub.set_index('session_id').to_csv("my_sub.csv")

**Обучите модель на всей выборке, сделайте прогноз для тестовой выборки и сделайте посылку в соревновании**.

In [27]:
# Ваш код здесь
write_to_submission_file(pred_proba, "sub_1.csv")

Если вы выполните эти действия и загрузите ответ на [странице](https://inclass.kaggle.com/c/catch-me-if-you-can-intruder-detection-through-webpage-session-tracking2) соревнования, то воспроизведете первый бенчмарк "Logit".

### 3. Улучшение модели, построение новых признаков

Создайте такой признак, который будет представлять собой число вида ГГГГММ от той даты, когда проходила сессия, например 201407 -- 2014 год и 7 месяц. Таким образом, мы будем учитывать помесячный [линейный тренд](http://people.duke.edu/~rnau/411trend.htm) за весь период предоставленных данных.

In [28]:
# Ваш код здесь

new_feat_train = pd.DataFrame(index=train_df.index)
new_feat_test = pd.DataFrame(index=test_df.index)

new_feat_train['year_month'] = train_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_df['time1'].apply(lambda ts: 100 * ts.year + ts.month)

Добавьте новый признак, предварительно отмасштабировав его с помощью `StandardScaler`, и снова посчитайте ROC AUC на отложенной выборке.

In [29]:
new_feat_train.shape, new_feat_test.shape

((253561, 1), (82797, 1))

In [30]:
# Ваш код здесь
scaler = StandardScaler()
scaler.fit(new_feat_train)

new_feat_train['year_month_scaled'] = scaler.transform(new_feat_train)
new_feat_test['year_month_scaled'] = scaler.transform(new_feat_test)

x_train_sparse.shape, new_feat_train['year_month_scaled'].shape

((253561, 48371), (253561,))

In [34]:
x_train_sparse_new = csr_matrix(hstack([x_train_sparse,
                                        new_feat_train[['year_month_scaled']]])) # ниже тоже сатое
x_test_sparse_new = csr_matrix(hstack([x_test_sparse,
                                        new_feat_test['year_month_scaled'].values.reshape(-1, 1)]))

x_train_sparse.shape, x_train_sparse_new.shape

((253561, 48371), (253561, 48372))

In [35]:
%%time
get_auc_lr_valid(x_train_sparse_new, y_train)

Wall time: 8.59 s


0.9196563464631973

**Добавьте два новых признака: start_hour и morning.**

Признак `start_hour` – это час в который началась сессия (от 0 до 23), а бинарный признак `morning` равен 1, если сессия началась утром и 0, если сессия началась позже (будем считать, что утро это если `start_hour равен` 11 или меньше).

**Посчитйте ROC AUC на отложенной выборке для выборки с:**
- сайтами, `start_month` и `start_hour`
- сайтами, `start_month` и `morning`
- сайтами, `start_month`, `start_hour` и `morning`

In [36]:
# Ваш код здесь
def f_start(ts):
    return ts.hour

new_feat_train['start'] = train_df['time1'].apply(f_start)
new_feat_test['start'] = test_df['time1'].apply(f_start)

In [37]:
new_feat_train.head()

,year_month,year_month_scaled,start
session_id,,,
21669,201301,-1.744405,8
54843,201301,-1.744405,8
77292,201301,-1.744405,8
114021,201301,-1.744405,8
146670,201301,-1.744405,8


In [40]:
col_names = ['start']
features = new_feat_train[col_names]

scaler = StandardScaler()
scaler.fit(features)

new_feat_train['start_scaled'] = scaler.transform(features.values)

features = new_feat_test[col_names]
new_feat_test['start_scaled'] = scaler.transform(features.values)
new_feat_train.head()

,year_month,year_month_scaled,start,start_scaled
session_id,,,,
21669,201301,-1.744405,8,-1.357366
54843,201301,-1.744405,8,-1.357366
77292,201301,-1.744405,8,-1.357366
114021,201301,-1.744405,8,-1.357366
146670,201301,-1.744405,8,-1.357366


In [39]:
x_train_sparse_new_s = csr_matrix(hstack([x_train_sparse_new,
                                        new_feat_train[['start_scaled']]]))
x_test_sparse_new_s = csr_matrix(hstack([x_test_sparse_new,
                                        new_feat_test[['start_scaled']]]))

x_train_sparse_new.shape, x_train_sparse_new_s.shape

((253561, 48372), (253561, 48373))

In [41]:
%%time
get_auc_lr_valid(x_train_sparse_new_s, y_train)

Wall time: 7.08 s


0.9572375929307422

### 4. Подбор коэффицициента регуляризации

Итак, мы ввели признаки, которые улучшают качество нашей модели по сравнению с первым бейслайном. Можем ли мы добиться большего значения метрики? После того, как мы сформировали обучающую и тестовую выборки, почти всегда имеет смысл подобрать оптимальные гиперпараметры -- характеристики модели, которые не изменяются во время обучения. Например, на 3 неделе вы проходили решающие деревья, глубина дерева это гиперпараметр, а признак, по которому происходит ветвление и его значение -- нет. В используемой нами логистической регрессии веса каждого признака изменяются и во время обучения находится их оптимальные значения, а коэффициент регуляризации остается постоянным. Это тот гиперпараметр, который мы сейчас будем оптимизировать.

Постараемся побить этот результат за счет оптимизации коэффициента регуляризации. Возьмем набор возможных значений C и для каждого из них посчитаем значение метрики на отложенной выборке.

Найдите `C` из `np.logspace(-3, 1, 10)`, при котором ROC AUC на отложенной выборке максимален. 

In [43]:
# Ваш код здесь
best_score, best_c = 0, None

for c in np.logspace(-3, 1, 10):
    score = get_auc_lr_valid(x_train_sparse_new_s, y_train, C=c)
    print(score, c)
    if score > best_score:
        best_score = score
        best_c = c
print()        
print("best_c: ", best_c);
print("best_score: ", best_score);

0.9329764195034997 0.001
0.9422026186059544 0.0027825594022071257
0.9513228440550894 0.007742636826811269
0.9583367299247858 0.021544346900318832
0.9613960775859987 0.05994842503189409
0.9612976519136849 0.1668100537200059
0.9592301089566153 0.46415888336127775
0.9568662322652639 1.2915496650148828
0.9547991421870548 3.593813663804626
0.9523439349255346 10.0

best_c:  0.05994842503189409
best_score:  0.9613960775859987


Наконец, обучите модель с найденным оптимальным значением коэффициента регуляризации и с построенными признаками `start_hour`, `start_month` и `morning`. Если вы все сделали правильно и загрузите это решение, то повторите второй бенчмарк соревнования.

In [46]:
# Ваш код здесь
model = LogisticRegression(C=best_c, random_state=21, n_jobs=-1)
model.fit(x_train_sparse_new_s, y_train)

pred_proba = model.predict_proba(x_test_sparse_new_s)[:, 1]
write_to_submission_file(pred_proba, "sub_2.csv")

In [47]:
x_train_sparse_new_s.shape, x_test_sparse_new_s.shape

((253561, 48373), (82797, 48373))

In [63]:
# !pip install eli5
import eli5

feature_names = [i for i in site2id.keys()] + ['start_scaled']
eli5.show_weights(estimator=model, 
                  feature_names= feature_names, top=30)

Weight?,Feature
+1.435,i1-js-14-3-01-10005-627271566-i.init.cedexis-radar.net
+1.276,www.circuit-electronique.fr
+1.142,milibris.com
+1.005,include.ebaystatic.com
+1.005,beemp3s.org
+0.963,start_scaled
+0.885,blog.planet-libre.org
+0.865,edutechwiki.unige.ch
+0.845,sliceofmit.files.wordpress.com
+0.830,auth.bay.livefilestore.com
